In [1]:
import os
# Set spark environments
os.environ['PYSPARK_PYTHON'] = r'C:\Users\room102sys2\AppData\Local\Programs\Python\Python39'
#os.environ['PYSPARK_DRIVER_PYTHON'] = r'C:\Users\room102sys2\AppData\Local\Programs\Python\Python39\Scripts\jupyter'
#os.environ['']

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
import findspark
findspark.init('C:\Program Files\Spark')

In [4]:
sp = SparkSession.builder.appName("SparkQL").getOrCreate()
sc = sp.sparkContext

# RESILENT DISTRIBUTED DATASETS

There are two sets of low-level APIs: 
- there is one for manipulating distributed data (RDDs), and
- another for distributing and manipulating distributed shared variables (broadcast variables and accumulators).

**When to Use the Low-Level APIs?**
- When we need some functionality that we cannot find in higer level APIs
- When we need to maintain legacy codebase written using RDDs
- When we need to do some custom shared variable manipulation 

**How to Use the Low-Level APIs?**
- A `SparkContext` is the entry point for low-level API functionality.
- We can access it through the `SparkSession`, which is the tool we use to perform computation across a Spark cluster.

In [ ]:
sc = sp.sparkContext

## About RDDs

- RDD represents an immutable, partitioned collection of records that can be operated on in parallel, unlike `DataFrame` through where each record is a structured row containing fields with known schema, in RDDs the records are just JAVA, Scala, or Python objects of the programmer's choosing.
- RDDs give us complete control because every record in an RDD is a just a Java or Python object.

## Types of RDDs
As a user, we will likely only be creating two types of RDDs:
- The generic 
- key-value i.e, that provieds additional functions, such as aggregating by key.

**each RDD is characterized by five main properties:**
- A list of partitions
- A function for computing each split
- A list of dependencies on other RDDs
- Optionally, a list of preferred locations on which to compute each split(e.g., block locations for a Hadoop Distributed File System [HDFS] file)
- Optionally, a `partitioner` for key-value RDDs (e.g., to say that the RDD is hash-partitioned)

**For Scala and Java, the performance is for the most part the same, the large costs incurred in manipulating the raw objects but in python can lose substantial amount of performance when using RDDs.**

Running Python RDDs equates to running Python user-defined functions (UDFs) row by row.

### When to use RDDs ?
The most likely reason for use RDDs is because we need fine-grained control over the physical distribution of data (custom partitioning of data)

## Creating RDDs
**